In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv(r"C:/data.csv", encoding="ISO-8859-1")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


knowing the number of rows and columns, the names of columns, data types, missing values, and duplicates.

In [2]:
df.shape

(541909, 8)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [4]:
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [5]:
df.duplicated().sum()

np.int64(5268)

Cleaning Stage.

-The results show that there are duplicate rows that should be removed to avoid negatively affecting the data. 

-Missing values can be dropped or filled, except for CustomerID, which does not impact the analysis. 

-Lastly, the InvoiceDate column will be converted to the datetime data type.

In [6]:
df.drop_duplicates(inplace=True)

Left the empty CustomerID columns empty because it does not have any real value

In [7]:
df = df[df['UnitPrice'] > 0]
df = df[df['Quantity'] > 0]

In [8]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

Analyzing stage.


Customer Analysis.

-Decide what “revenue” means

-Analyze customers

-Ranking

-Revenue concentration

In [9]:
df['Revenue'] = df['Quantity'] * df['UnitPrice']

Made a new column calculating the total money spent per item by multiplying the Quantity columns by the UnitPrice column.

In [10]:
df.groupby('CustomerID')['Revenue'].sum().sort_values(ascending=False)

CustomerID
14646.0    280206.02
18102.0    259657.30
17450.0    194390.79
16446.0    168472.50
14911.0    143711.17
             ...    
16878.0        13.30
17956.0        12.75
16454.0         6.90
14792.0         6.20
16738.0         3.75
Name: Revenue, Length: 4338, dtype: float64

That calulated the total money spent per customer using the Revenue column that i created and CustomerID.

In [11]:
print(df.groupby('CustomerID')['Revenue'].sum().sort_values(ascending=False).head(1).sum() / df['Revenue'].sum() * 100)
print(1 / df['CustomerID'].nunique())

print(df.groupby('CustomerID')['Revenue'].sum().sort_values(ascending=False).head(5).sum() / df['Revenue'].sum() * 100)
print(5 / df['CustomerID'].nunique())

print(df.groupby('CustomerID')['Revenue'].sum().sort_values(ascending=False).head(10).sum() / df['Revenue'].sum() * 100)
print(10 / df['CustomerID'].nunique())

2.6329928823394724
0.0002305209774089442
9.832990834926097
0.001152604887044721
14.448817892612501
0.002305209774089442


I took the total percent of revenue from 1, 5, and 10 customers and compared that to what percent they are from the total customers, and that showed that:


This indicates that the top spender accounts for 2.6% of all revenue, which is a significant portion of revenue for only 0.00023% of customers.


This indicates that the top 10 spenders account for 9.8% of all revenue, which is a significant portion of revenue for only 0.0011% of customers.


This indicates that the top 10 spenders account for 14.4% of all revenue, which is a significant portion of revenue for only 0.0023% of customers.

In [12]:
print(df.groupby('CustomerID')['Revenue'].sum().sort_values(ascending=False).head(int(0.2 * df['CustomerID'].nunique())).sum() / df['Revenue'].sum() * 100)

62.34895908531644


This shows that the top 20 percent of customers account for 62% of revenue.

Product Analysis.

Now, I will examine the top products based on their total revenue and total quantity sold, as the price of a product doesn't indicate the actual profit made. We will consider both factors.

In [16]:
df.groupby('StockCode').agg(Revenue=('Revenue', 'sum'), Description=('Description', 'first')).sort_values('Revenue', ascending=False)

,Revenue,Description
StockCode,,
DOT,206248.770,DOTCOM POSTAGE
22423,174156.540,REGENCY CAKESTAND 3 TIER
23843,168469.600,"PAPER CRAFT , LITTLE BIRDIE"
85123A,104462.750,WHITE HANGING HEART T-LIGHT HOLDER
47566,99445.230,PARTY BUNTING
...,...,...
90084,0.850,PINK CRYSTAL GUITAR PHONE CHARM
21268,0.840,VINTAGE BLUE TINSEL REEL
51014c,0.830,"FEATHER PEN,COAL BLACK"


This calculated the total revenue for every product(StockCode) and provided the description of that item.

In [18]:
df.groupby('StockCode').agg(Quantity=('Quantity', 'sum'), Description=('Description', 'first')).sort_values('Quantity', ascending=False)

,Quantity,Description
StockCode,,
23843,80995,"PAPER CRAFT , LITTLE BIRDIE"
23166,78033,MEDIUM CERAMIC TOP STORAGE JAR
22197,56898,SMALL POPCORN HOLDER
84077,54951,WORLD WAR 2 GLIDERS ASSTD DESIGNS
85099B,48371,JUMBO BAG RED RETROSPOT
...,...,...
90187A,1,BLUE DROP EARRINGS W BEAD CLUSTER
90181C,1,BLACK GLASS/SHELL/PEARL NECKLACE
90178B,1,PURPLE CHUNKY GLASS+BEAD NECKLACE


This calculates the total quantity sold for every item along with its description. We can see that the items in this list are not the same as in the revenue list.

In [30]:
df.groupby('StockCode')['Revenue'].sum().sort_values(ascending=False).head(int(0.2 * df['StockCode'].nunique())).sum() / df['Revenue'].sum() * 100

np.float64(79.47113543321834)

This tells us thet the top 20% of all products amount to 80% of all profit.